In [3]:
# imports
import pandas as pd
import numpy as np

# file path
DATA_PATH = "C:/Users/502731688/Desktop/DA/PROJECT 2/data/raw/"

file_match_stats = DATA_PATH + "epl_2324_match_stats.csv"
file_player_stats = DATA_PATH + "premier-player-23-24.csv"
file_projectsem = DATA_PATH + "projectsem5dataset.csv"
file_pl_stats = DATA_PATH + "premier-league.csv"

df_match23 = pd.read_csv(file_match_stats, encoding="utf-8-sig")
df_players23 = pd.read_csv(file_player_stats, encoding="utf-8-sig")
df_proj = pd.read_csv(file_projectsem, encoding="latin1")
df_pl = pd.read_csv(file_pl_stats, encoding="utf-8-sig")

In [4]:
# preview
for name, df in {
    "Match Stats 23/24": df_match23,
    "Players 23/24": df_players23,
    "ProjectSem Dataset": df_proj,
    "PL Stats": df_pl  
}.items():
    print(f"\n--- {name} ---")
    print(df.shape)
    print(df.head(3))


--- Match Stats 23/24 ---
(380, 27)
   Unnamed: 0  round           start_time    home_team          away_team  \
0           0      1  2023/08/11 14:00:00      Burnley    Manchester City   
1           1      1  2023/08/12 07:00:00      Arsenal  Nottingham Forest   
2           2      1  2023/08/12 09:00:00  Bournemouth    West Ham United   

   home_score  away_score  home_possession  away_possession  home_shots  ...  \
0           0           3             0.34             0.66           6  ...   
1           2           1             0.78             0.22          15  ...   
2           1           1             0.63             0.37          14  ...   

   home_fouls  away_fouls  home_yellow_cards  away_yellow_cards  \
0          11           8                  0                  0   
1          12          12                  2                  2   
2           9          14                  1                  4   

   home_red_cards  away_red_cards  home_offsides  away_offsides 

In [5]:
# cleaning players23
df_p = df_players23.copy()
# column names
df_p.columns = (
    df_p.columns.str.lower().str.strip().str.replace(" ", "_")
)
# team/club
df_p.rename(columns = {"team": "club"}, inplace = True)
# adding seasonID
df_p["season"] = "2023/24"
# create playerID
import unicodedata
import re

def normalize_name(text):
    # if NaN/None → NaN or empty string
    if pd.isna(text):
        return np.nan
    text = str(text)
    # remove diakritika
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    # lowercase
    text = text.lower()
    text = re.sub(r'[^a-z0-9 ]+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df_p["player_clean"] = df_p["player"].apply(normalize_name)
df_p["club_clean"] = df_p["club"].apply(normalize_name)

#df_p["player_id"] = (
#   df_p["player_clean"].str.replace(" ", "_") +
#    "_" +
#    df_p["club_clean"].str.replace(" ", "_")
#)

# per 90 calculation

df_p["goals_per90"] = df_p["gls"] / df_p["90s"].replace(0, np.nan)
df_p["xa_per90"] = df_p["ast"] / df_p["90s"].replace(0, np.nan)
df_p["ga_per90"] = (df_p["gls"] + df_p["ast"]) / df_p["90s"].replace(0, np.nan)

In [6]:
# preview special characters
df_p[df_p["player"].str.contains("lv")] \
    [["player", "club"]].head(5)

,player,club
3,Julián Álvarez,Manchester City
5,Bernardo Silva,Manchester City
20,Kalvin Phillips,Manchester City
85,Thiago Silva,Chelsea
243,Edson Álvarez,West Ham United


In [7]:
df_p.head()

,player,nation,pos,age,mp,starts,min,90s,gls,ast,...,xg+xag_90,npxg_90,npxg+xag_90,club,season,player_clean,club_clean,goals_per90,xa_per90,ga_per90
0,Rodri,es ESP,MF,27.0,34,34,2931.0,32.6,8.0,9.0,...,0.24,0.12,0.24,Manchester City,2023/24,rodri,manchester city,0.245399,0.276074,0.521472
1,Phil Foden,eng ENG,"FW,MF",23.0,35,33,2857.0,31.7,19.0,8.0,...,0.59,0.33,0.59,Manchester City,2023/24,phil foden,manchester city,0.599369,0.252366,0.851735
2,Ederson,br BRA,GK,29.0,33,33,2785.0,30.9,0.0,0.0,...,0.00,0.00,0.00,Manchester City,2023/24,ederson,manchester city,0.000000,0.000000,0.000000
3,Julián Álvarez,ar ARG,"MF,FW",23.0,36,31,2647.0,29.4,11.0,8.0,...,0.66,0.39,0.61,Manchester City,2023/24,julian alvarez,manchester city,0.374150,0.272109,0.646259
4,Kyle Walker,eng ENG,DF,33.0,32,30,2767.0,30.7,0.0,4.0,...,0.10,0.01,0.10,Manchester City,2023/24,kyle walker,manchester city,0.000000,0.130293,0.130293


In [8]:
# position mapping
def map_position(pos):
    p = str(pos).lower()
    if "gk" in p:
        return "GK"
    if "fw" in p or "st" in p:
        return "ATT"
    if "mf" in p or "mid" in p:
        return "MID"
    if "df" in p or "cb" in p or "lb" in p or "rb" in p:
        return "DEF"
    return "UNK"

df_p["pos_group"] = df_p["pos"].apply(map_position)

# output index
df_p["goals_per90"] = df_p["goals_per90"].fillna(0)
df_p["xg_90"] = df_p["xg_90"].fillna(0)
df_p["xag_90"] = df_p["xag_90"].fillna(0)

# normalize metrics [scale 0-1]
def normalize(s):
    return (s - s.min()) / (s.max() - s.min() + 1e-9)
df_p["goals_norm"] = normalize(df_p["goals_per90"])
df_p["xg_norm"] = normalize(df_p["xg_90"])
df_p["xag_norm"] = normalize(df_p["xag_90"])

# output index (ATT - focused)
df_p["output_index_v0"] = (
    0.45 * df_p["goals_norm"] +
    0.2 * df_p["xg_norm"] +
    0.35 * df_p["xag_norm"]
)

# set index = 0 for GK (just temporary)
df_p.loc[df_p["pos_group"] == "GK", "output_index_v0"] = 0

In [9]:
MIN_MINUTES = 1500  # threshold 900 min. played

df_candidates = (
    df_p[df_p["min"] >= MIN_MINUTES]
    .copy()
)

df_att = df_candidates[df_candidates["pos_group"] == "ATT"]

df_att_top10 = df_att.sort_values(
    "output_index_v0", ascending=False
).head(10)[
    ["player", "club", "pos", "min",
     "goals_per90", "xg_90", "xag_90",
     "output_index_v0"]
]

df_att_top10

,player,club,pos,min,goals_per90,xg_90,xag_90,output_index_v0
6,Erling Haaland,Manchester City,FW,2552.0,0.950704,1.03,0.15,0.246728
117,Alexander Isak,Newcastle United,FW,2255.0,0.836653,0.81,0.15,0.212577
83,Cole Palmer,Chelsea,"FW,MF",2607.0,0.758621,0.63,0.38,0.205516
28,Mohamed Salah,Liverpool,FW,2534.0,0.638298,0.75,0.42,0.194441
458,Chris Wood,Nottingham Forest,FW,1812.0,0.696517,0.59,0.10,0.169789
65,Leandro Trossard,Arsenal,"FW,MF",1649.0,0.655738,0.43,0.15,0.156482
32,Darwin Núñez,Liverpool,FW,2047.0,0.484581,0.72,0.27,0.153090
265,Jean-Philippe Mateta,Crystal Palace,FW,2282.0,0.629921,0.43,0.12,0.149471
1,Phil Foden,Manchester City,"FW,MF",2857.0,0.599369,0.33,0.26,0.148815
146,Son Heung-min,Tottenham Hotspur,FW,2934.0,0.521472,0.37,0.36,0.145154


In [10]:
# wages load
df_w = pd.read_csv(DATA_PATH + "soccer_salaries.csv")
# normalize columns names
df_w.columns = (
    df_w.columns.str.lower()
                .str.strip()
                .str.replace(" ", "_")
)
# clean currency from weekly wage
df_w["weekly_wage"] = (
    df_w["weekly_wage"]
    .astype(str)
    .str.replace("£", "", regex=False)
    .str.replace(",", "", regex=False)
)
df_w["weekly_wage"] = pd.to_numeric(df_w["weekly_wage"], errors="coerce")
# same for yearly salary
df_w["yearly_salary"] = (
    df_w["yearly_salary"]
    .astype(str)
    .str.replace("£", "", regex=False)
    .str.replace(",", "", regex=False)
)
df_w["yearly_salary"] = pd.to_numeric(df_w["yearly_salary"], errors="coerce")
df_w["annual_wage"] = df_w["yearly_salary"]
# debugging
team_map = {
    "afc-bournemouth": "bournemouth",
    "manchester-city-f.c.": "manchester city",
    "manchester-united-f.c.": "manchester united",
    "tottenham-hotspur-f.c.": "tottenham hotspur",
    "west-ham-united-f.c.": "west ham united",
    "wolverhampton-wanderers-f.c.": "wolverhampton",
    "nottingham-forest":"nottingham forest",
    "brighton-&-hove-albion": "brighton",
    "sheffield-united-f.c.": "sheffield utd",
    "luton-town": "luton town",
    "arsenal-f.c.": "arsenal",
    "brentford": "brentford",
    "burnley-f.c.": "burnley",
    "chelsea-f.c.": "chelsea",
    "crystal-palace": "crystal palace",
    "everton-f.c.": "everton",
    "fulham": "fulham",
    "liverpool-f.c.": "liverpool",
    "newcastle-united-f.c.": "newcastle united",
    "aston-villa-f.c.": "aston villa"
}
df_w["team_norm"] = (
    df_w["team"]
      .str.lower()
      .str.strip()
      .replace(team_map)
)
# normalize names for join
df_w["player_clean"] = df_w["player_name"].apply(normalize_name)
df_w["club_clean"] = df_w["team_norm"].apply(normalize_name)
# merge to df_p
df_p = df_p.merge(
    df_w[["player_clean", "club_clean", "annual_wage"]],
    on = ["player_clean", "club_clean"],
    how = "left"
)
# missings -> median
median_wage = df_p["annual_wage"].median()
df_p["annual_wage"] = df_p["annual_wage"].fillna(median_wage)

In [11]:
# transfers (load & clean)
df_tr = df_pl.copy()
# normalize
df_tr.columns = (
    df_tr.columns.str.lower().str.strip().str.replace(" ", "_")
)
# filtering
df_tr = df_tr[
    (df_tr["league_name"] == "Premier League") &
    (df_tr["transfer_movement"] == "in")].copy()
# clean  + numberic fee/deleting loans
df_tr["player_clean"] = df_tr["player_name"].apply(normalize_name)
df_tr["club_clean"] = df_tr["club_name"].apply(normalize_name)
df_tr["transfer_fee"] = pd.to_numeric(df_tr["fee_cleaned"], errors = "coerce").fillna(0)
df_tr = df_tr[~df_tr["fee"].str.contains("End of loan", na=False)]
# just highest/newest transfer
df_tr_player_club = (
    df_tr
    .sort_values(["player_clean", "club_clean", "year"], ascending = [True, True, False])
    .groupby(["player_clean", "club_clean"], as_index = False)
    .agg({"transfer_fee": "max"})
)
# merge transfers to players_table
df_p = df_p.merge(
    df_tr_player_club[["player_clean", "club_clean", "transfer_fee"]],
    on = ["player_clean", "club_clean"],
    how = "left"
)
df_p["transfer_fee"] = df_p["transfer_fee"].fillna(0) # missing tranfer details -> 0
df_p["amortized_fee"] = df_p["transfer_fee"] / 5 # amortize 5 yrs

In [12]:
# cost index v2
df_p["cost_index_raw"] = df_p["annual_wage"] + df_p["amortized_fee"]
df_p["cost_index_v2"] = normalize(df_p["cost_index_raw"]) #normalization 0-1
df_p["value_index_v2"] = df_p["output_index_v0"] / (df_p["cost_index_v2"] + 1e-6) # value index v2

In [13]:
# best value forwards (23/24) 
df_value = df_p[df_p["min"] >= MIN_MINUTES].copy()
df_att_value = df_value[df_value["pos_group"] == "ATT"].copy()
df_att_value = df_att_value.drop_duplicates(subset=["player", "club"]) # duplicates removed
df_att_bestvalue = df_att_value.sort_values(
    "value_index_v2",
    ascending = False
).head(10)[
    ["player", "club", "pos", "min",
     "goals_per90", "xg_90", "xag_90",
     "annual_wage", "transfer_fee",
     "output_index_v0", "cost_index_v2", "value_index_v2"]
]

In [14]:
df_p["transfer_fee_corrected"] = df_p["transfer_fee"].replace(0, 5.0)  # 5m € floor
df_p["amortized_fee"] = df_p["transfer_fee_corrected"] / 5

In [15]:
# === MARKET VALUE INTEGRATION (Transfermarkt) === #

# load transfermarkt market value
df_tm_players = pd.read_csv(DATA_PATH + "player_profiles.csv", low_memory=False)
df_tm_mv = pd.read_csv(DATA_PATH + "player_market_value.csv")

# normalize column names
df_tm_players.columns = df_tm_players.columns.str.lower().str.replace(" ", "_")
df_tm_mv.columns = df_tm_mv.columns.str.lower().str.replace(" ", "_")

print("df_tm_mv columns:", df_tm_mv.columns.tolist())

# 1) find column with date
if "date" in df_tm_mv.columns:
    date_col = "date"
elif "date_unix" in df_tm_mv.columns:
    date_col = "date_unix"
else:
    raise ValueError("Nenašiel som stĺpec s dátumom (ani 'date', ani 'date_unix').")

# just one datetime column "date"
df_tm_mv["date"] = pd.to_datetime(df_tm_mv[date_col], errors="coerce")

# 2) find column market value
value_candidates = [c for c in df_tm_mv.columns if "value" in c]
print("Value-like columns in df_tm_mv:", value_candidates)

if "market_value" in df_tm_mv.columns:
    value_col = "market_value"
elif "value" in df_tm_mv.columns:
    value_col = "value"
elif value_candidates:
    value_col = value_candidates[0]   # just first one (if more)
else:
    raise ValueError("Nenašiel som žiadny stĺpec s 'value' v názve v df_tm_mv.")

# filter 23/24
season_start = pd.Timestamp("2023-07-01")
season_end = pd.Timestamp("2024-06-30")

df_tm_2324 = df_tm_mv[
    (df_tm_mv["date"] >= season_start) &
    (df_tm_mv["date"] <= season_end)
].copy()

print("df_tm_2324 shape:", df_tm_2324.shape)

# last value MV per time player_id
df_tm_2324 = (
    df_tm_2324
    .sort_values(["player_id", "date"])
    .groupby("player_id", as_index=False)
    .tail(1)
    .rename(columns={value_col: "market_value"})
)

print("df_tm_2324 MV columns:", [c for c in df_tm_2324.columns if "value" in c])

# TM players names
df_tm_players["player_clean"] = df_tm_players["player_name"].apply(normalize_name)
# remove stick ID z clean name "james milner 3333"
df_tm_players["player_clean"] = (
    df_tm_players["player_clean"]
    .str.replace(r"\d+", "", regex=True)  # remove all numbers
    .str.replace(r"\s+", " ", regex=True)  # reduction of spaces
    .str.strip()
)
# join MV with names cez player_id - df_tm_full
df_tm_full = df_tm_2324.merge(
    df_tm_players[["player_id", "player_clean"]],
    on="player_id",
    how="left"
)

# debugging
df_tm_name_mv = (
    df_tm_full
    .groupby("player_clean", as_index=False)
    .agg({"market_value": "max"})   # alebo "mean" / "last" podľa preferencie
)
print(df_tm_name_mv.head(15))

# one row per player_clean → highest market_value
df_tm_name_mv = (
    df_tm_full
    .groupby("player_clean", as_index=False)
    .agg({"market_value": "max"})
)

print(df_tm_name_mv.head(10))

print("df_tm_full sample:")
print(df_tm_full[["player_id", "player_clean", "market_value"]].head(10))

# 5) odstránime staré market_value stĺpce z df_p (ak by tam niečo zostalo)
mv_cols_in_p = [c for c in df_p.columns if "market_value" in c]
print("Removing old MV columns from df_p:", mv_cols_in_p)
df_p = df_p.drop(columns=mv_cols_in_p, errors="ignore")


# remove old MV columns (if anything there)
mv_cols_in_p = [c for c in df_p.columns if "market_value" in c]
df_p = df_p.drop(columns=mv_cols_in_p, errors="ignore")
# new merge - 1 value MV na 1 player_clean - duplicated rows
df_p = df_p.merge(
    df_tm_name_mv[["player_clean", "market_value"]],
    on="player_clean",
    how="left"
)

print("Columns in df_p after MV merge:", [c for c in df_p.columns if "market" in c or "value" in c])

print("Non-null market_value ratio:", df_p["market_value"].notna().mean())
print(
    df_p[df_p["club"] == "Manchester City"]
    [["player", "player_clean", "market_value"]]
    .head(15)
)

# 7) Fill missing market values with median as fallback
df_p["market_value"] = df_p["market_value"].fillna(df_p["market_value"].median())

# create market value-based cost index
df_p["cost_mv_raw"] = df_p["market_value"]
df_p["cost_mv"] = normalize(df_p["cost_mv_raw"])

# value index v3: output / market value
df_p["value_index_mv"] = df_p["output_index_v0"] / (df_p["cost_mv"] + 1e-6)

df_tm_mv columns: ['player_id', 'date_unix', 'value']
Value-like columns in df_tm_mv: ['value']
df_tm_2324 shape: (90811, 4)
df_tm_2324 MV columns: ['market_value']
             player_clean  market_value
0              aapo halme      150000.0
1         aaron anselmino     3000000.0
2            aaron appiah      200000.0
3      aaron appindangoye      800000.0
4          aaron basenach      100000.0
5            aaron berzel       50000.0
6            aaron bibout       75000.0
7         aaron boupendza     5000000.0
8         aaron cervantes           0.0
9   aaron ciammaglichella     1000000.0
10          aaron collins     1300000.0
11         aaron connolly     2500000.0
12         aaron cosgrave           0.0
13        aaron cresswell      900000.0
14         aaron donnelly      500000.0
            player_clean  market_value
0             aapo halme      150000.0
1        aaron anselmino     3000000.0
2           aaron appiah      200000.0
3     aaron appindangoye      800000.0


In [16]:
    dups = df_p[df_p.duplicated(subset=["player_clean", "club_clean"], keep=False)]

dups[["player_clean", "club", "min", "annual_wage", "transfer_fee", "market_value"]].sort_values(
    ["player_clean", "club", "min"], ascending=[True, True, False]
).head(30)

,player_clean,club,min,annual_wage,transfer_fee,market_value
458,aaron hickey,Brentford,713.0,1820000.0,0.0,28000000.0
459,aaron hickey,Brentford,713.0,1820000.0,0.0,28000000.0
449,ben mee,Brentford,1272.0,2236000.0,0.0,1000000.0
450,ben mee,Brentford,1272.0,2236000.0,0.0,1000000.0
441,bryan mbeumo,Brentford,1960.0,2236000.0,0.0,40000000.0
442,bryan mbeumo,Brentford,1960.0,2236000.0,0.0,40000000.0
429,christian nrgaard,Brentford,2505.0,2184000.0,0.0,18000000.0
430,christian nrgaard,Brentford,2505.0,2184000.0,0.0,18000000.0
435,ethan pinnock,Brentford,2521.0,1664000.0,0.0,12000000.0
436,ethan pinnock,Brentford,2521.0,1664000.0,0.0,12000000.0


In [17]:
#remove duplicates
df_p_unique = (
    df_p
    .sort_values(["player_clean", "club_clean", "min"], ascending=[True, True, False])
    .drop_duplicates(subset=["player_clean", "club"], keep="first")
    .reset_index(drop=True)
)

In [18]:
df_p_unique = (
    df_p
    .sort_values(["player_clean", "club_clean", "min"], ascending=[True, True, False])
    .drop_duplicates(subset=["player_clean", "club"], keep="first")
    .reset_index(drop=True)
)
df_p = df_p_unique

In [19]:
df_p[df_p["player_clean"] == "alisson"][["player", "club", "min", "annual_wage", "market_value"]]

,player,club,min,annual_wage,market_value
25,Alisson,Liverpool,2520.0,11700000.0,28000000.0


In [20]:
df_p[df_p["club"] == "Manchester City"][["player", "market_value"]].head(15)

,player,market_value
59,Aymeric Laporte,20000000.0
75,Bernardo Silva,70000000.0
121,Cole Palmer,80000000.0
165,Ederson,40000000.0
178,Erling Haaland,180000000.0
231,Jack Grealish,60000000.0
249,James Mcatee,12000000.0
271,Jeremy Doku,65000000.0
292,John Stones,38000000.0
306,Joško Gvardiol,75000000.0


In [21]:
# best value forwards — market valuev3
df_best_mv = df_p[
    (df_p["pos_group"] == "ATT") &
    (df_p["min"] >= MIN_MINUTES)
].copy()

df_top_mv = (
    df_best_mv
    .sort_values("value_index_mv", ascending=False)
    .head(15)
    [["player", "club", "min", "market_value", 
      "goals_per90", "output_index_v0", "value_index_mv"]]
)

df_top_mv

,player,club,min,market_value,goals_per90,output_index_v0,value_index_mv
114,Chris Wood,Nottingham Forest,1812.0,7000000.0,0.696517,0.169789,4.365878
400,Michail Antonio,West Ham United,1695.0,4000000.0,0.319149,0.080583,3.626085
568,Willian,Fulham,2053.0,3000000.0,0.175439,0.058602,3.515917
138,Danny Welbeck,Brighton,1693.0,5000000.0,0.265957,0.072304,2.602865
295,Jordan Ayew,Crystal Palace,2543.0,4000000.0,0.141343,0.045935,2.066971
518,Son Heung-min,Tottenham Hotspur,2934.0,13000000.0,0.521472,0.145154,2.009792
217,Hwang Hee-chan,Wolverhampton,2119.0,13000000.0,0.510638,0.122596,1.697461
48,Antoine Semenyo,Bournemouth,2105.0,10000000.0,0.341880,0.084958,1.529224
454,Pablo Sarabia,Wolverhampton,1745.0,10000000.0,0.206186,0.078100,1.405766
267,Jean-Philippe Mateta,Crystal Palace,2282.0,20000000.0,0.629921,0.149471,1.345224


In [22]:
# === BASE FILTER pre value rebríčky ===
MIN_MINUTES = 1500

df_value_base = df_p[
    (df_p["pos_group"] == "ATT") &
    (df_p["min"] >= MIN_MINUTES)
].copy()

# odstránime istotu duplikáty (pre istotu, keby boli z mergov)
df_value_base = df_value_base.drop_duplicates(subset=["player", "club"])

# --- 1) Hlavný ranking: Best Value Forwards (bez limitu market value) ---

df_best_all = (
    df_value_base
    .sort_values("value_index_mv", ascending=False)
    .head(20)[
        ["player", "club", "min",
         "market_value",
         "goals_per90", "xg_90", "xag_90",
         "output_index_v0", "value_index_mv"]
    ]
)

print("TOP 20 Best Value Forwards – ALL market values")
print(df_best_all.to_string(index=False))


# --- 2) Vedľajší ranking: Best Value Forwards (MV <= 60M) ---

MV_MAX = 60000000  # 60 miliónov €

df_best_u60 = (
    df_value_base[df_value_base["market_value"] <= MV_MAX]
    .sort_values("value_index_mv", ascending=False)
    .head(20)[
        ["player", "club", "min",
         "market_value",
         "goals_per90", "xg_90", "xag_90",
         "output_index_v0", "value_index_mv"]
    ]
)

print("\nTOP 20 Best Value Forwards – market_value <= 60M")
print(df_best_u60.to_string(index=False))

TOP 20 Best Value Forwards – ALL market values
              player              club    min  market_value  goals_per90  xg_90  xag_90  output_index_v0  value_index_mv
          Chris Wood Nottingham Forest 1812.0     7000000.0     0.696517   0.59    0.10         0.169789        4.365878
     Michail Antonio   West Ham United 1695.0     4000000.0     0.319149   0.31    0.05         0.080583        3.626085
             Willian            Fulham 2053.0     3000000.0     0.175439   0.22    0.17         0.058602        3.515917
       Danny Welbeck          Brighton 1693.0     5000000.0     0.265957   0.28    0.09         0.072304        2.602865
         Jordan Ayew    Crystal Palace 2543.0     4000000.0     0.141343   0.14    0.15         0.045935        2.066971
       Son Heung-min Tottenham Hotspur 2934.0    13000000.0     0.521472   0.37    0.36         0.145154        2.009792
      Hwang Hee-chan     Wolverhampton 2119.0    13000000.0     0.510638   0.33    0.13         0.122596  

In [23]:
# BASE FILTER 
MIN_MINUTES = 1500

df_value_base = df_p[
    (df_p["pos_group"] == "ATT") &
    (df_p["min"] >= MIN_MINUTES)
].copy()

# drop duplicates
df_value_base = df_value_base.drop_duplicates(subset=["player", "club"])

# definicion MV
LOW_MAX = 10_000_000      # <= 10M
MID_MAX = 60_000_000      # 10–60M
# > 60M = elite

def mv_tier(v):
    if v <= LOW_MAX:
        return "Low (<= 10M €)"
    elif v <= MID_MAX:
        return "Mid (10M–60M €)"
    else:
        return "Elite (> 60M €)"

df_value_base["mv_tier"] = df_value_base["market_value"].apply(mv_tier)

# 1) Low-tier bargains 
df_low = (
    df_value_base[df_value_base["mv_tier"] == "Low (<= 10M €)"]
    .sort_values("value_index_mv", ascending=False)
    .head(15)[
        ["player", "club", "min",
         "market_value", "goals_per90", "xg_90", "xag_90",
         "output_index_v0", "value_index_mv"]
    ]
)

print("\n### Best Value Forwards – LOW tier (<= 10M €)")
print(df_low.to_string(index=False))

# 2) Mid-tier smart deals
df_mid = (
    df_value_base[df_value_base["mv_tier"] == "Mid (10M–60M €)"]
    .sort_values("value_index_mv", ascending=False)
    .head(15)[
        ["player", "club", "min",
         "market_value", "goals_per90", "xg_90", "xag_90",
         "output_index_v0", "value_index_mv"]
    ]
)

print("\n### Best Value Forwards – MID tier (10M–60M €)")
print(df_mid.to_string(index=False))

# 3) Elite-tier efficiency 
df_elite = (
    df_value_base[df_value_base["mv_tier"] == "Elite (> 60M €)"]
    .sort_values("value_index_mv", ascending=False)
    .head(15)[
        ["player", "club", "min",
         "market_value", "goals_per90", "xg_90", "xag_90",
         "output_index_v0", "value_index_mv"]
    ]
)

print("\n### Best Value Forwards – ELITE tier (> 60M €)")
print(df_elite.to_string(index=False))


### Best Value Forwards – LOW tier (<= 10M €)
            player              club    min  market_value  goals_per90  xg_90  xag_90  output_index_v0  value_index_mv
        Chris Wood Nottingham Forest 1812.0     7000000.0     0.696517   0.59    0.10         0.169789        4.365878
   Michail Antonio   West Ham United 1695.0     4000000.0     0.319149   0.31    0.05         0.080583        3.626085
           Willian            Fulham 2053.0     3000000.0     0.175439   0.22    0.17         0.058602        3.515917
     Danny Welbeck          Brighton 1693.0     5000000.0     0.265957   0.28    0.09         0.072304        2.602865
       Jordan Ayew    Crystal Palace 2543.0     4000000.0     0.141343   0.14    0.15         0.045935        2.066971
   Antoine Semenyo       Bournemouth 2105.0    10000000.0     0.341880   0.20    0.14         0.084958        1.529224
     Pablo Sarabia     Wolverhampton 1745.0    10000000.0     0.206186   0.28    0.30         0.078100        1.405766
A

In [24]:
# SOFT COST: logaritmic MV penalization 
# cost_mv_raw = market_value -> raw cost
# cost_mv_log_raw = log(1 + market_value) -> soft cost

df_p["cost_mv_log_raw"] = np.log1p(df_p["market_value"]) # !!!!
df_p["cost_mv_log"] = normalize(df_p["cost_mv_log_raw"])
df_p["value_index_mv_log"] = df_p["output_index_v0"] / (df_p["cost_mv_log"] + 1e-6)

In [25]:
df_value_base_log = df_p[
    (df_p["pos_group"] == "ATT") &
    (df_p["min"] >= MIN_MINUTES)
].copy()

df_value_base_log = df_value_base_log.drop_duplicates(subset=["player", "club"])

df_best_log = (
    df_value_base_log
    .sort_values("value_index_mv_log", ascending=False)
    .head(20)[
        ["player", "club", "min",
         "market_value",
         "goals_per90", "xg_90", "xag_90",
         "output_index_v0", "value_index_mv_log"]
    ]
)

print("\n Best Value Forwards – SOFT cost (log MV)")
print(df_best_log.to_string(index=False))


 Best Value Forwards – SOFT cost (log MV)
              player              club    min  market_value  goals_per90  xg_90  xag_90  output_index_v0  value_index_mv_log
      Erling Haaland   Manchester City 2552.0   180000000.0     0.950704   1.03    0.15         0.246728            0.246728
      Alexander Isak  Newcastle United 2255.0    75000000.0     0.836653   0.81    0.15         0.212577            0.222840
         Cole Palmer           Chelsea 2607.0    80000000.0     0.758621   0.63    0.38         0.205516            0.214674
       Mohamed Salah         Liverpool 2534.0    55000000.0     0.638298   0.75    0.42         0.194441            0.207376
          Chris Wood Nottingham Forest 1812.0     7000000.0     0.696517   0.59    0.10         0.169789            0.204767
    Leandro Trossard           Arsenal 1649.0    35000000.0     0.655738   0.43    0.15         0.156482            0.171234
Jean-Philippe Mateta    Crystal Palace 2282.0    20000000.0     0.629921   0.43   

In [26]:
print(df_p.columns.tolist())

['player', 'nation', 'pos', 'age', 'mp', 'starts', 'min', '90s', 'gls', 'ast', 'g+a', 'g-pk', 'pk', 'pkatt', 'crdy', 'crdr', 'xg', 'npxg', 'xag', 'npxg+xag', 'prgc', 'prgp', 'prgr', 'gls_90', 'ast_90', 'g+a_90', 'g-pk_90', 'g+a-pk_90', 'xg_90', 'xag_90', 'xg+xag_90', 'npxg_90', 'npxg+xag_90', 'club', 'season', 'player_clean', 'club_clean', 'goals_per90', 'xa_per90', 'ga_per90', 'pos_group', 'goals_norm', 'xg_norm', 'xag_norm', 'output_index_v0', 'annual_wage', 'transfer_fee', 'amortized_fee', 'cost_index_raw', 'cost_index_v2', 'value_index_v2', 'transfer_fee_corrected', 'market_value', 'cost_mv_raw', 'cost_mv', 'value_index_mv', 'cost_mv_log_raw', 'cost_mv_log', 'value_index_mv_log']


In [31]:
df_p["age"] = df_players23["Age"]

In [32]:
# adding "Age"
if "age" not in df_p.columns:
    # Try common alternative names
    possible_age_cols = [col for col in df_p.columns if "age" in col.lower()]
    
    if possible_age_cols:
        df_p.rename(columns={possible_age_cols[0]: "age"}, inplace=True)
    else:
        raise ValueError("Age column not found in df_p. Please check original dataset.")

df_p["age"] = pd.to_numeric(df_p["age"], errors="coerce")

In [33]:
df_p["age"]

0      27.0
1      23.0
2      29.0
3      23.0
4      33.0
       ... 
575    17.0
576    19.0
577    19.0
578    18.0
579    17.0
Name: age, Length: 580, dtype: float64

In [34]:
# TIER filtering with updated df_p (includes age)
df_low = df_p[
    (df_p["pos_group"] == "ATT") &
    (df_p["min"] >= 1500) &
    (df_p["market_value"] <= 10_000_000)
]

df_mid = df_p[
    (df_p["pos_group"] == "ATT") &
    (df_p["min"] >= 1500) &
    (df_p["market_value"] > 10_000_000) &
    (df_p["market_value"] <= 60_000_000)
]

df_elite = df_p[
    (df_p["pos_group"] == "ATT") &
    (df_p["min"] >= 1500) &
    (df_p["market_value"] > 60_000_000)
]

# columns we want
cols_value = [
    "player", "club", "age", "min",
    "market_value",
    "goals_per90", "xg_90", "xag_90",
    "output_index_v0", "value_index_mv"
]

# create display views
df_low_view = df_low[cols_value].sort_values("value_index_mv", ascending=False).head(20)
df_mid_view = df_mid[cols_value].sort_values("value_index_mv", ascending=False).head(20)
df_elite_view = df_elite[cols_value].sort_values("value_index_mv", ascending=False).head(20)


In [35]:
# 1) Znovu načítaj pôvodný players 23/24 dataset (zdroj veku)
file_player_stats = DATA_PATH + "premier-player-23-24.csv"
df_players23_src = pd.read_csv(file_player_stats, encoding="utf-8-sig")

# 2) Zosúladíme stĺpce ako v df_p
df_players23_src.columns = (
    df_players23_src.columns.str.lower()
                             .str.strip()
                             .str.replace(" ", "_")
)

# 3) Vytvoríme rovnaký player_clean ako v df_p
import unicodedata, re

def normalize_name(text):
    if pd.isna(text):
        return ""
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    text = text.lower()
    text = re.sub(r'[^a-z0-9 ]+', '', text)
    text = text.strip()
    return text

df_players23_src["player_clean"] = df_players23_src["player"].apply(normalize_name)

# 4) Vytiahneme vek z tohto zdroja
ages_src = df_players23_src[["player_clean", "age"]].copy()

# 5) Vyhodíme rozbitý age v df_p a nahradíme ho týmto
df_p = df_p.drop(columns=["age"], errors="ignore")

df_p = df_p.merge(
    ages_src,
    on="player_clean",
    how="left"
)

In [36]:

print("\n Best Value – LOW tier with Age")
print(df_low_view.to_string(index=False))

print("\n Best Value – MID tier with Age")
print(df_mid_view.to_string(index=False))

print("\n Best Value – ELITE tier with Age")
print(df_elite_view.to_string(index=False))


 Best Value – LOW tier with Age
            player              club  age    min  market_value  goals_per90  xg_90  xag_90  output_index_v0  value_index_mv
        Chris Wood Nottingham Forest 22.0 1812.0     7000000.0     0.696517   0.59    0.10         0.169789        4.365878
   Michail Antonio   West Ham United 30.0 1695.0     4000000.0     0.319149   0.31    0.05         0.080583        3.626085
           Willian            Fulham 22.0 2053.0     3000000.0     0.175439   0.22    0.17         0.058602        3.515917
     Danny Welbeck          Brighton 18.0 1693.0     5000000.0     0.265957   0.28    0.09         0.072304        2.602865
       Jordan Ayew    Crystal Palace 25.0 2543.0     4000000.0     0.141343   0.14    0.15         0.045935        2.066971
   Antoine Semenyo       Bournemouth 18.0 2105.0    10000000.0     0.341880   0.20    0.14         0.084958        1.529224
     Pablo Sarabia     Wolverhampton 21.0 1745.0    10000000.0     0.206186   0.28    0.30         

In [37]:
df_p.loc[df_p["player"] == "Cole Palmer", ["player", "club", "age"]]

,player,club,age
128,Cole Palmer,Chelsea,21.0
129,Cole Palmer,Chelsea,21.0
130,Cole Palmer,Manchester City,21.0
131,Cole Palmer,Manchester City,21.0


In [38]:
# debug
# TIER filtering with updated df_p (includes age)
df_value_base = df_p[
    (df_p["pos_group"] == "ATT") &
    (df_p["min"] >= MIN_MINUTES)
].copy()
df_value_base = df_value_base.drop_duplicates(subset=["player", "club"])

df_low = df_value_base[df_value_base["market_value"] <= 10_000_000]
df_mid = df_value_base[(df_value_base["market_value"] > 10_000_000) & (df_value_base["market_value"] <= 60_000_000)]
df_elite = df_value_base[df_value_base["market_value"] > 60_000_000]



# columns we want
cols_value = [
    "player", "club", "age", "min",
    "market_value",
    "goals_per90", "xg_90", "xag_90",
    "output_index_v0", "value_index_mv"
]


# create display views
df_low_view = df_low[cols_value].sort_values("value_index_mv", ascending=False).head(20)
df_mid_view = df_mid[cols_value].sort_values("value_index_mv", ascending=False).head(20)
df_elite_view = df_elite[cols_value].sort_values("value_index_mv", ascending=False).head(20)

In [39]:
print("\n Best Value – LOW tier with Age")
print(df_low_view.to_string(index=False))

print("\n Best Value – MID tier with Age")
print(df_mid_view.to_string(index=False))

print("\n Best Value – ELITE tier with Age")
print(df_elite_view.to_string(index=False))


 Best Value – LOW tier with Age
            player              club  age    min  market_value  goals_per90  xg_90  xag_90  output_index_v0  value_index_mv
        Chris Wood Nottingham Forest 31.0 1812.0     7000000.0     0.696517   0.59    0.10         0.169789        4.365878
   Michail Antonio   West Ham United 33.0 1695.0     4000000.0     0.319149   0.31    0.05         0.080583        3.626085
           Willian            Fulham 34.0 2053.0     3000000.0     0.175439   0.22    0.17         0.058602        3.515917
     Danny Welbeck          Brighton 32.0 1693.0     5000000.0     0.265957   0.28    0.09         0.072304        2.602865
       Jordan Ayew    Crystal Palace 31.0 2543.0     4000000.0     0.141343   0.14    0.15         0.045935        2.066971
   Antoine Semenyo       Bournemouth 23.0 2105.0    10000000.0     0.341880   0.20    0.14         0.084958        1.529224
     Pablo Sarabia     Wolverhampton 31.0 1745.0    10000000.0     0.206186   0.28    0.30         

In [40]:
df_p.to_csv("players_value_model.csv", index=False)

In [41]:
export_cols = [
    "player", "club","age", "pos", "pos_group",
    "min",
    "market_value",
    "goals_per90", "xg_90", "xag_90",
    "output_index_v0",
    "value_index_mv",      # hard cost
    "value_index_mv_log"   # soft cost
]

df_export = df_p[export_cols].copy()

df_export.to_csv("players_value_model_2324.csv", index=False)


In [43]:
def assign_tier(mv):
    if pd.isna(mv):
        return "UNKNOWN"
    elif mv <= 10_000_000:
        return "LOW"
    elif mv <= 60_000_000:
        return "MID"
    else:
        return "ELITE"

df_p["price_tier"] = df_p["market_value"].apply(assign_tier)


cols_export = [
    "player", "club", "age", "min",
    "market_value", "price_tier",
    "goals_per90", "xg_90", "xag_90",
    "output_index_v0",
    "value_index_mv", "value_index_mv_log"
]

df_export = df_p[df_p["pos_group"] == "ATT"][cols_export].copy()

df_export.to_csv(
    "players_value_model_2324_v2.csv",
    index=False,
    float_format="%.6f",
    encoding="utf-8"
)

In [46]:
df_export.to_csv(
    "players_value_model_2324_v2.csv",
    index=False,
    float_format="%.6f",
    encoding="utf-8"
)